https://medium.com/@curiousily/tensorflow-for-hackers-part-ii-building-simple-neural-network-2d6779d2f91b

In [ ]:
import pandas as pd
from math import floor
import tensorflow as tf
import numpy as np
from sklearn import preprocessing

In [ ]:
data1 = pd.read_csv('premier_14_15.csv')
data2 = pd.read_csv('premier_15_16.csv')
data3 = pd.read_csv('premier_16_17.csv')
data4 = pd.read_csv('premier_17_18.csv')

dataCon = [data1,data2,data3,data4]
data = pd.concat(dataCon)

In [ ]:
b = np.random.uniform(low=-0.01,high=0.01,size=(3,3))
a = pd.DataFrame(b,columns=['a','b','c'])
a

In [ ]:
def normalization(raw_data):
    for col_num in range(raw_data.shape[1]):
        if raw_data.iloc[:,col_num].dtype == np.float or raw_data.iloc[:,col_num].dtype == np.int:
            raw_data.iloc[:,col_num] = (raw_data.iloc[:,col_num] - raw_data.iloc[:,col_num].mean()) / (raw_data.iloc[:,col_num].max() - raw_data.iloc[:,col_num].min())
    return raw_data

In [ ]:
def embedding_matrix(column):
    labels = []
    embeddings = np.array([])
    num_of_uniques = len(np.unique(column))
    for i in range(num_of_uniques):
        if embeddings.size == 0:
            embeddings = np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1))
        else:
            embeddings = np.append(embeddings,np.random.uniform(low=-0.01,high=0.01,size=(min(50,(num_of_uniques+1)//2),1)),axis=1)
        labels.append(np.unique(column)[i])
    print("embeddings.shape:",embeddings.shape)
    print("labels.shape:",len(labels))
    return pd.DataFrame(data=embeddings,columns=labels)

In [ ]:
em = embedding_matrix(data['HomeTeam'])
em.head()

In [ ]:
#x_data = data.drop(['FTR','FTAG','FTHG'],1)
data = normalization(data)
x_data = np.column_stack((np.transpose(em[data['HomeTeam']].values),
                          np.transpose(em[data['AwayTeam']].values),
                          data['B365H'][:,None],
                          data['B365D'][:,None],
                          data['B365A'][:,None]))
y_data = data['FTR']

y_data = pd.get_dummies(y_data)

#differences = data['FTHG']-data['FTAG']

In [ ]:
x_data.shape

In [ ]:
# Two first games
x_data[:,0:2]

In [ ]:
y_data.shape

In [ ]:
y_data.head()

In [ ]:
train_size = 0.9
valid_size = 0.3

train_cnt = floor(x_data.shape[0] * train_size)

x_train = x_data[0:train_cnt]
y_train = y_data.iloc[0:train_cnt].values

valid_cnt = floor((x_data.shape[0] - train_cnt) * valid_size)

x_valid = x_data[train_cnt:train_cnt+valid_cnt]
y_valid = y_data.iloc[train_cnt:train_cnt+valid_cnt].values

x_test = x_data[train_cnt+valid_cnt:]
y_test = y_data.iloc[train_cnt+valid_cnt:]

print("x_train:",x_train.shape)
print("x_valid:",x_valid.shape)
print("x_test:",x_test.shape)

In [ ]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

In [ ]:
# Parameters
LEARNING_RATE_STA = 7.5e-2
num_epochs = 30
batch_size = 128
display_step = 5

In [ ]:
# Network Parameters
num_input = x_data.shape[1]
num_classes = y_data.shape[1]
num_hidden_1 = 50
num_hidden_2 = 50
num_hidden_3 = 50
KEEP_PROB_STA = 0.7

In [ ]:
def neural_network(x,weights,biases,keep_prob):
    layer_1 = tf.add(tf.matmul(x,weights['w1']),biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1,keep_prob)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['w2']),biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_2 = tf.nn.dropout(layer_2,keep_prob)
    
    layer_3 = tf.add(tf.matmul(layer_2,weights['w3']),biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_3 = tf.nn.dropout(layer_3,keep_prob)
    
    layer_out = tf.add(tf.matmul(layer_3, weights['out']), biases['out'])
    return layer_out

In [ ]:
# Store layers weight & bias
weights = {
    'w1': tf.Variable(tf.random_normal([num_input,num_hidden_1])),
    'w2': tf.Variable(tf.random_normal([num_hidden_1,num_hidden_2])),
    'w3': tf.Variable(tf.random_normal([num_hidden_2,num_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_hidden_3, num_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([num_hidden_1])),
    'b2': tf.Variable(tf.random_normal([num_hidden_2])),
    'b3': tf.Variable(tf.random_normal([num_hidden_3])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

keep_prob = tf.placeholder("float")

In [ ]:
predictions = neural_network(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE_STA).minimize(cost)

In [ ]:
costs = []
lrn_rate_sizes = []

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(num_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: KEEP_PROB_STA
                            })
            avg_cost += c / total_batch
            
        if epoch % display_step == 0:
            print("Train: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(avg_cost))
            _, valid_cost = sess.run([optimizer, cost], feed_dict={x: x_valid, y: y_valid, keep_prob: 1})
            print("Valid: Epoch:", '%04d' % (epoch+display_step), "cost=", "{:.9f}".format(valid_cost))

            prediction=tf.argmax(predictions,1)
            print("Predictions:")
            print("")
            print("")
            print("")
            print(prediction.eval(feed_dict={x: x_test, y: y_test, keep_prob: 1.0}))
            print("")
            print("")
            print("")
            print("$$$$$$$$$$$$$$$$$$$$$$$$")
            
            correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
            print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))
            print("______________________________________")
    print("Optimization Finished!")
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))